In [46]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
from IPython.display import Audio
import wave
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
root_fp = "../../DATA/ml-stuttering-events-dataset/"
CLIPS_DIR = "/home/alien/Git/DATA/ml-stuttering-events-dataset/CLIP_DIR/"
df = pd.read_csv(root_fp + "SEP-28k_labels.csv")
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [48]:
df.shape

(28177, 17)

In [49]:
df['Name'] = df[df.columns[0:3]].apply(
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech,Name
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_1
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_2
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0,HeStutters_0_3
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_4


In [50]:
df['Name']

0                HeStutters_0_0
1                HeStutters_0_1
2                HeStutters_0_2
3                HeStutters_0_3
4                HeStutters_0_4
                  ...          
28172    WomenWhoStutter_109_35
28173    WomenWhoStutter_109_36
28174    WomenWhoStutter_109_37
28175    WomenWhoStutter_109_38
28176    WomenWhoStutter_109_39
Name: Name, Length: 28177, dtype: object

In [51]:
for dire in os.listdir(CLIPS_DIR):
    for sub in os.listdir(CLIPS_DIR + dire):
        for f in os.listdir(CLIPS_DIR + dire + "/" + sub):
            fp = CLIPS_DIR + dire + "/" + sub + "/" + f
            if 'FluencyBank' not in f:
                if os.stat(fp).st_size == 44:
                    print(fp)
# no empty?

In [52]:
df.shape

(28177, 18)

In [53]:
features = {}
for d in os.listdir(CLIPS_DIR):
    for subd in os.listdir(CLIPS_DIR + d):
        for filename in os.listdir(CLIPS_DIR + d + "/" + subd):
            temp = filename[:-4]
            fp = CLIPS_DIR + d + "/" + subd + "/" + filename
            audio, sample_rate = librosa.load(fp, res_type='kaiser_fast', sr=None)
            mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13).T,axis=0)
            features[temp] = mfccs

In [55]:
print(len(features))
# first_key = next(iter(features))
# first_value = features[first_key]

# print(first_key, first_value)
print(features['MyStutteringLife_20_4'])

21856
[-432.12228    112.04518    -36.72194     38.552074    -9.890725
  -37.270924     4.442986   -18.82082    -11.932086    -2.5179555
  -17.764147    -2.4562576   -3.481152 ]


In [56]:
df_features = pd.DataFrame.from_dict(features)
df_features = df_features.transpose()
df_features

,0,1,2,3,4,5,6,7,8,9,10,11,12
MyStutteringLife_20_4,-432.122284,112.045181,-36.721939,38.552074,-9.890725,-37.270924,4.442986,-18.820820,-11.932086,-2.517956,-17.764147,-2.456258,-3.481152
MyStutteringLife_20_19,-391.077057,87.192001,-40.239815,24.385504,-9.296728,-36.503204,-3.359330,-15.630564,-12.132483,-2.883445,-9.625459,-3.057173,-3.170635
MyStutteringLife_20_3,-369.965393,107.020370,-61.755402,13.552789,-22.951670,-36.735798,-1.921751,-19.177439,-7.748105,-4.050750,-6.974270,1.646603,-8.805808
MyStutteringLife_20_28,-453.750000,94.437813,-53.773640,23.920488,-19.514503,-25.909538,-1.486993,-26.949291,-7.905697,-7.212661,-11.114883,1.461078,-7.268161
MyStutteringLife_20_8,-404.244141,103.642494,-34.330116,-7.386903,-13.031321,-19.330818,0.838191,-15.218096,-13.309039,0.619506,-4.869978,-6.122318,-5.590073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
IStutterSoWhat_4_32,-369.166962,123.694878,11.809272,31.433018,14.693367,14.663195,6.293732,5.702393,5.646729,-5.192612,3.966316,8.825089,-0.381951
IStutterSoWhat_4_138,-278.272369,74.952087,-10.659211,56.537373,-19.393883,12.835714,-5.411568,9.976400,-10.657762,8.029886,-10.353887,2.980437,-1.755309
IStutterSoWhat_4_96,-353.455048,133.776779,9.270531,40.139229,11.387088,13.678348,2.558310,1.187103,-1.339023,-7.014803,2.958161,3.871079,-2.055159
IStutterSoWhat_4_58,-335.568451,137.070648,5.925931,29.025908,10.636375,18.107809,2.916996,4.554673,4.551122,-7.557528,3.667790,4.749824,-0.963060


In [57]:
df_features = df_features.reset_index()
df_features

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,MyStutteringLife_20_4,-432.122284,112.045181,-36.721939,38.552074,-9.890725,-37.270924,4.442986,-18.820820,-11.932086,-2.517956,-17.764147,-2.456258,-3.481152
1,MyStutteringLife_20_19,-391.077057,87.192001,-40.239815,24.385504,-9.296728,-36.503204,-3.359330,-15.630564,-12.132483,-2.883445,-9.625459,-3.057173,-3.170635
2,MyStutteringLife_20_3,-369.965393,107.020370,-61.755402,13.552789,-22.951670,-36.735798,-1.921751,-19.177439,-7.748105,-4.050750,-6.974270,1.646603,-8.805808
3,MyStutteringLife_20_28,-453.750000,94.437813,-53.773640,23.920488,-19.514503,-25.909538,-1.486993,-26.949291,-7.905697,-7.212661,-11.114883,1.461078,-7.268161
4,MyStutteringLife_20_8,-404.244141,103.642494,-34.330116,-7.386903,-13.031321,-19.330818,0.838191,-15.218096,-13.309039,0.619506,-4.869978,-6.122318,-5.590073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21851,IStutterSoWhat_4_32,-369.166962,123.694878,11.809272,31.433018,14.693367,14.663195,6.293732,5.702393,5.646729,-5.192612,3.966316,8.825089,-0.381951
21852,IStutterSoWhat_4_138,-278.272369,74.952087,-10.659211,56.537373,-19.393883,12.835714,-5.411568,9.976400,-10.657762,8.029886,-10.353887,2.980437,-1.755309
21853,IStutterSoWhat_4_96,-353.455048,133.776779,9.270531,40.139229,11.387088,13.678348,2.558310,1.187103,-1.339023,-7.014803,2.958161,3.871079,-2.055159
21854,IStutterSoWhat_4_58,-335.568451,137.070648,5.925931,29.025908,10.636375,18.107809,2.916996,4.554673,4.551122,-7.557528,3.667790,4.749824,-0.963060


In [58]:
df_features = df_features.sort_values(by='index')
df_features

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
2537,HVSA_0_0,-672.897095,42.572784,-9.060439,17.092854,5.601506,11.969574,5.087627,1.287422,-1.221630,5.906602,0.396098,2.618646,2.006739
2629,HVSA_0_1,-463.573730,69.182426,-18.914116,22.859167,0.735867,-7.253482,-7.202471,-5.319007,-5.854908,4.434800,-2.941299,0.067582,-10.047692
2586,HVSA_0_10,-452.379944,58.461617,-26.529619,10.806691,1.040473,-11.684329,-15.975926,-17.110899,-8.533467,3.804462,-0.904730,1.546427,-5.672744
2571,HVSA_0_100,-454.134918,85.565826,-16.623894,23.212719,-7.808586,-7.204660,-8.445378,-17.406126,-16.752789,-5.885397,0.342330,1.163907,-5.125211
2542,HVSA_0_101,-419.757965,73.650085,-20.263416,30.691788,-5.919299,-15.747584,-17.799961,-13.380462,-10.132165,-2.647574,1.943859,-0.861621,-5.781014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20918,WomenWhoStutter_9_5,-331.060883,8.423851,-32.912979,25.618399,-23.380960,2.367096,-14.927188,-5.307941,0.028746,-7.770761,-5.659317,-3.837930,-1.988690
20943,WomenWhoStutter_9_6,-287.171539,24.612055,11.593957,-2.373049,-19.041893,1.381284,-8.410904,-6.094374,-21.232731,1.682852,-18.440504,5.257711,-14.749590
20939,WomenWhoStutter_9_7,-284.419769,69.935936,4.447401,25.512430,-23.518784,-3.872571,2.403355,-14.279772,-22.612999,5.954081,-23.500135,-1.513236,-22.199059
20927,WomenWhoStutter_9_8,-359.677887,48.367851,14.129713,3.370583,-19.779791,1.081605,-1.616947,-4.654747,-21.594097,-1.051946,-16.132671,8.522419,-16.694715


In [59]:
df_features['index']

2537                HVSA_0_0
2629                HVSA_0_1
2586               HVSA_0_10
2571              HVSA_0_100
2542              HVSA_0_101
                ...         
20918    WomenWhoStutter_9_5
20943    WomenWhoStutter_9_6
20939    WomenWhoStutter_9_7
20927    WomenWhoStutter_9_8
20907    WomenWhoStutter_9_9
Name: index, Length: 21856, dtype: object

In [61]:
df['Name']

0                HeStutters_0_0
1                HeStutters_0_1
2                HeStutters_0_2
3                HeStutters_0_3
4                HeStutters_0_4
                  ...          
28172    WomenWhoStutter_109_35
28173    WomenWhoStutter_109_36
28174    WomenWhoStutter_109_37
28175    WomenWhoStutter_109_38
28176    WomenWhoStutter_109_39
Name: Name, Length: 28177, dtype: object

In [62]:
df_features.rename(columns = {'index':'Name'}, inplace = True)
df_features

,Name,0,1,2,3,4,5,6,7,8,9,10,11,12
2537,HVSA_0_0,-672.897095,42.572784,-9.060439,17.092854,5.601506,11.969574,5.087627,1.287422,-1.221630,5.906602,0.396098,2.618646,2.006739
2629,HVSA_0_1,-463.573730,69.182426,-18.914116,22.859167,0.735867,-7.253482,-7.202471,-5.319007,-5.854908,4.434800,-2.941299,0.067582,-10.047692
2586,HVSA_0_10,-452.379944,58.461617,-26.529619,10.806691,1.040473,-11.684329,-15.975926,-17.110899,-8.533467,3.804462,-0.904730,1.546427,-5.672744
2571,HVSA_0_100,-454.134918,85.565826,-16.623894,23.212719,-7.808586,-7.204660,-8.445378,-17.406126,-16.752789,-5.885397,0.342330,1.163907,-5.125211
2542,HVSA_0_101,-419.757965,73.650085,-20.263416,30.691788,-5.919299,-15.747584,-17.799961,-13.380462,-10.132165,-2.647574,1.943859,-0.861621,-5.781014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20918,WomenWhoStutter_9_5,-331.060883,8.423851,-32.912979,25.618399,-23.380960,2.367096,-14.927188,-5.307941,0.028746,-7.770761,-5.659317,-3.837930,-1.988690
20943,WomenWhoStutter_9_6,-287.171539,24.612055,11.593957,-2.373049,-19.041893,1.381284,-8.410904,-6.094374,-21.232731,1.682852,-18.440504,5.257711,-14.749590
20939,WomenWhoStutter_9_7,-284.419769,69.935936,4.447401,25.512430,-23.518784,-3.872571,2.403355,-14.279772,-22.612999,5.954081,-23.500135,-1.513236,-22.199059
20927,WomenWhoStutter_9_8,-359.677887,48.367851,14.129713,3.370583,-19.779791,1.081605,-1.616947,-4.654747,-21.594097,-1.051946,-16.132671,8.522419,-16.694715


In [63]:
df_final = pd.merge(df, df_features, how='inner', on='Name')
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
21852,WomenWhoStutter,109,36,26168480,26216480,0,0,1,1,3,...,29.588446,-10.896355,-14.222120,-9.167952,-1.676774,-8.374956,-6.297094,1.265485,-4.014944,-0.196149
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21854,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [64]:
df_final.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355


In [67]:
df_final.info()
df_final.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21856 entries, 0 to 21855
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Show                   21856 non-null  object 
 1   EpId                   21856 non-null  int64  
 2   ClipId                 21856 non-null  int64  
 3   Start                  21856 non-null  int64  
 4   Stop                   21856 non-null  int64  
 5   Unsure                 21856 non-null  int64  
 6   PoorAudioQuality       21856 non-null  int64  
 7   Prolongation           21856 non-null  int64  
 8   Block                  21856 non-null  int64  
 9   SoundRep               21856 non-null  int64  
 10  WordRep                21856 non-null  int64  
 11  DifficultToUnderstand  21856 non-null  int64  
 12  Interjection           21856 non-null  int64  
 13  NoStutteredWords       21856 non-null  int64  
 14  NaturalPause           21856 non-null  int64  
 15  Mu

(21856, 31)

In [68]:
df_final = df_final[df_final.PoorAudioQuality == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
21852,WomenWhoStutter,109,36,26168480,26216480,0,0,1,1,3,...,29.588446,-10.896355,-14.222120,-9.167952,-1.676774,-8.374956,-6.297094,1.265485,-4.014944,-0.196149
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21854,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [69]:
df_final = df_final[df_final.DifficultToUnderstand == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21850,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831638,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21854,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [70]:
df_final = df_final[df_final.Music == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21850,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831638,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21854,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [71]:
df_final = df_final[df_final.NoSpeech == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21850,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831638,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21854,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [72]:
df_final.to_csv("./sep28k-mfcc.csv",index=False)

In [87]:
df = pd.read_csv("./sep28k-mfcc.csv")

In [88]:
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574424,-7.091357,-1.978344,-8.984579,-4.724132,-3.345275,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926200,-1.934813,-8.946336,-8.399076
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762988,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803762,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16412,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831638,-24.096672,-9.137759,-8.294925,-17.235230,-3.540464,-5.074892,-3.828139,-5.524148,-5.444070
16413,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143099,-26.160053,3.709429,-15.719260,2.436124,-1.615821,-5.105482,-10.272832,-3.074109,-6.375842
16414,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942732,-1.638087,-3.736744,-0.968271,-7.699818,0.135875
16415,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132100,-4.451934,-1.437802,-11.729773,-6.692394,0.137587,-3.664662


In [89]:
df = df[df['NoStutteredWords'] != 0]
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['Prolongation'] == 0]
df = df[df['WordRep'] == 0]
df = df[df['Block'] == 0]
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803762,-6.036355
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,4.350783,-14.230149,-3.446103,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,HeStutters,0,9,41417440,41465440,0,0,0,0,0,...,10.802255,0.253166,-0.571133,-7.632134,-2.497033,-8.864999,-2.741920,-5.094120,-9.865230,-3.816170
11,HeStutters,0,11,42928800,42976800,0,0,0,0,0,...,24.965216,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16402,WomenWhoStutter,109,24,16179040,16227040,0,0,0,0,0,...,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517434,1.568335,-6.449186,1.097754
16403,WomenWhoStutter,109,25,16224160,16272160,0,0,0,0,0,...,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16409,WomenWhoStutter,109,31,20476320,20524320,0,0,0,0,0,...,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16411,WomenWhoStutter,109,33,24747360,24795360,0,0,0,0,0,...,18.849430,-23.086039,2.158884,-8.471071,1.332329,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [90]:
df.loc[df['SoundRep'] >= 1.0, 'SoundRep'] = 1.0
df['SoundRep'].value_counts()

SoundRep
0    1704
1     121
Name: count, dtype: int64

In [91]:
X = df.iloc[: , -13:]
X


,0,1,2,3,4,5,6,7,8,9,10,11,12
2,-313.66693,62.418663,-1.334004,36.037710,-2.650919,-11.021186,-9.310137,-1.562137,-13.035163,-8.655547,-4.336684,-7.372598,-7.138989
4,-341.35330,68.858000,-3.959719,8.430685,-0.561030,-4.473868,-9.220818,-3.201562,-11.855061,-4.699772,-2.414847,-7.803762,-6.036355
6,-351.60030,79.161500,10.344581,4.350783,-14.230149,-3.446103,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,-322.53464,84.011630,-0.464125,10.802255,0.253166,-0.571133,-7.632134,-2.497033,-8.864999,-2.741920,-5.094120,-9.865230,-3.816170
11,-310.75836,27.546114,-17.378850,24.965216,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16402,-396.81607,47.699245,-28.230492,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517434,1.568335,-6.449186,1.097754
16403,-366.09630,59.089397,-23.645912,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16409,-316.00030,42.245895,-40.056310,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16411,-350.42902,27.453814,-35.155937,18.849430,-23.086039,2.158884,-8.471071,1.332329,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [92]:
y = df['SoundRep']
y

2        0
4        0
6        0
9        0
11       0
        ..
16402    0
16403    0
16409    0
16411    0
16416    0
Name: SoundRep, Length: 1825, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.4, random_state=42)

from collections import Counter
Counter(y_train)

Counter({0: 1028, 1: 67})

In [94]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini',random_state=5)
# clf.fit(X_over, y_over)
# clf.fit(X_smote, y_smote)
# clf.fit(X_pipe, y_pipe)
clf.fit(X_train, y_train)
clf.score(X_test,y_test) * 100

86.3013698630137

In [95]:
y_pred = np.array(clf.predict(X_test))
y_actual = np.array(y_test)
print(Counter(y_pred))
print(Counter(y_actual))

Counter({0: 684, 1: 46})
Counter({0: 676, 1: 54})
